In [3]:
import datetime
import pandas as pd
import numpy as np

#테스트 파일 로딩
df_org = pd.read_csv('./data/XRPUSDT_index.csv')
print(df_org.shape)
print(df_org.columns)

(168883, 30)
Index(['t', 'o', 'h', 'l', 'c', 'v', 'sma3', 'sma5', 'sma7', 'sma25', 'sma99',
       'ema7', 'ema25', 'ema99', 'macd', 'macd_s', 'macd_d', 'rsi6', 'rsi12',
       'rsi24', 'srsi', 'srsik', 'srsid', 'bh', 'bhi', 'bl', 'bli', 'bm', 'bw',
       'vwap'],
      dtype='object')


In [11]:
#타임스켐프 시간 변환
def get_time_hhmmss(mili_time):
    mili_time = float(mili_time)/1000
    KST = datetime.timezone(datetime.timedelta(hours=9))
    dt = datetime.datetime.fromtimestamp(mili_time, tz=KST)
    timeline = str(dt.strftime('%D %H:%M:%S'))
    return timeline

#단계별 구매 수량
def get_open_amt_list(open_amt_unit, open_cnt_limit, increace_rate):
    open_amt = 0
    open_amt_list = [0.0]
    for idx in range(0, open_cnt_limit):
        temp_amt = open_amt_unit + open_amt * increace_rate
        open_amt = round(open_amt + temp_amt, 4)
        open_amt_list.append(open_amt)
    return open_amt_list

#손실 최소화 실현 금액 계산
def get_max_loss(close, open_amt_unit, open_cnt_limit, increace_rate, max_loss_rate):
    open_amt = 0
    open_price = 0
    for idx in range(0, open_cnt_limit):
        temp_amt = open_amt_unit + open_amt * increace_rate
        open_price = round(open_price + close * temp_amt, 4)
        open_amt = round(open_amt + temp_amt, 4)
    return round(open_price * max_loss_rate, 4)

revenue_rate = 0.011  #익절 비율
max_loss_rate = 0.18   #손절 비율: 20%
increace_rate = 0.25  
open_cnt_limit = 7   #최대 오픈 건수
open_amt_unit = 9     #최소 오픈 수량
close = 0.6
open_amt_list = get_open_amt_list(open_amt_unit, open_cnt_limit, increace_rate)
max_loss = get_max_loss(close, open_amt_unit, open_cnt_limit, increace_rate, max_loss_rate)
print("*open_amt_list:{} max_loss:{}".format(open_amt_list, max_loss))

open_long_cnt = 0 
open_long_price = 0
open_long_amt = 0
open_short_cnt = 0
open_short_price = 0
open_short_amt = 0
revenue = 0
revenue_t = 0

df = df_org.iloc[df_org.shape[0]-144000:,]
# df = df_org
for i in range(0, df.shape[0]-1):
    
    close1 = round(df.iloc[i:i+1,]['c'].values[0],4)
    close2 = round(df.iloc[i+1:i+2,]['c'].values[0],4)
    vwap = round(df.iloc[i:i+1,]['vwap'].values[0],4)
    t = round(df.iloc[i+1:i+2,]['t'].values[0],4)
    
    #long position stop loss
    long_loss = open_long_price - close2*open_long_amt
    if long_loss > max_loss:
        revenue_t = close2*open_long_amt - open_long_price
        revenue = round(revenue + revenue_t,4)
        print("*[{}] time:{} long stop loss:{} revenue:{} ".format(i, get_time_hhmmss(t), revenue_t, revenue))        
        open_long_cnt = 0
        open_long_amt = 0
        open_long_price = 0
        continue
            
    #short position stop loss
    short_loss = close2*open_short_amt - open_short_price
    if short_loss > max_loss:
        revenue_t = open_short_price - close2*open_short_amt
        revenue = round(revenue + revenue_t,4)
        print("*[{}] time:{} short stop loss:{} revenue:{}".format(i, get_time_hhmmss(t), revenue_t, revenue))         
        open_short_cnt = 0
        open_short_amt = 0        
        open_short_price = 0
        momentum = ''
        continue
    
    #long take profit
    long_revenue = close2*open_long_amt - \
                        (open_long_price + open_long_price*revenue_rate)
    if open_long_cnt > 0 and long_revenue > 0:
        revenue_t = close2*open_long_amt - open_long_price
        revenue = round(revenue + revenue_t,4)
        open_long_cnt = 0
        open_long_amt = 0
        open_long_price = 0
        continue
    
    #short take profit
    short_revenue = open_short_price - \
                        (close2*open_short_amt + close2*open_short_amt*revenue_rate)
    if open_short_cnt > 0 and short_revenue > 0:
        revenue_t = open_short_price - close2*open_short_amt
        revenue = round(revenue + revenue_t,4)
        open_short_cnt = 0
        open_short_amt = 0        
        open_short_price = 0
        continue
    
    #long position open
    if open_long_cnt < open_cnt_limit and close1 < vwap:
        temp_amt = open_amt_unit + open_long_amt*increace_rate
        open_long_price = round(open_long_price + close2*temp_amt,4)
        open_long_amt = round(open_long_amt + temp_amt,4)
        open_long_cnt = open_long_cnt + 1 

    #short position open        
    if open_short_cnt < open_cnt_limit and close1 > vwap:
        temp_amt = open_amt_unit + open_short_amt*increace_rate
        open_short_price = round(open_short_price + close2*temp_amt,4)
        open_short_amt = round(open_short_amt + temp_amt,4)
        open_short_cnt = open_short_cnt + 1  
        
#     print("c1:{} c2:{} vwap:{} olc:{} ola:{} olp:{} osc:{} osa:{} osp:{} revenue:{}"
#           .format(close1, close2, vwap, open_long_cnt, open_long_amt, open_long_price, 
#                   open_short_cnt, open_short_amt, open_short_price, revenue))

print("*revenue:{}".format(revenue))

*open_amt_list:[0.0, 9.0, 20.25, 34.3125, 51.8906, 73.8632, 101.329, 135.6612] max_loss:14.6514
*[7488] time:03/21/21 07:25:00 short stop loss:-15.077760560000002 revenue:-6.9167
*[13246] time:03/25/21 07:17:00 long stop loss:-16.13190759999999 revenue:-6.7377 
*[20808] time:03/30/21 13:12:00 short stop loss:-14.943163760000004 revenue:-0.8324
*[29311] time:04/05/21 10:46:00 short stop loss:-14.814941200000007 revenue:3.8382
*[30154] time:04/06/21 00:48:00 short stop loss:-14.814898880000001 revenue:-4.253
*[30204] time:04/06/21 01:38:00 short stop loss:-15.486154920000004 revenue:-14.5985
*[30494] time:04/06/21 06:28:00 short stop loss:-15.173590320000002 revenue:-21.2683
*[31150] time:04/06/21 17:23:00 short stop loss:-14.922790800000001 revenue:-20.5644
*[31276] time:04/06/21 19:29:00 short stop loss:-15.18395012000002 revenue:-28.9879
*[31477] time:04/06/21 22:50:00 long stop loss:-16.21475124 revenue:-35.9675 
*[31839] time:04/07/21 04:52:00 short stop loss:-15.382149640000009 rev

*[86387] time:05/15/21 01:05:00 short stop loss:-15.04183236 revenue:180.3423
*[87490] time:05/15/21 19:27:00 long stop loss:-15.107442239999983 revenue:183.698 
*[87736] time:05/15/21 23:33:00 short stop loss:-15.667715399999992 revenue:174.4551
*[88023] time:05/16/21 04:19:00 long stop loss:-15.056681239999989 revenue:173.5148 
*[88161] time:05/16/21 06:37:00 short stop loss:-15.37155292 revenue:163.6061
*[88520] time:05/16/21 12:36:00 short stop loss:-15.170781760000011 revenue:170.1827
*[89226] time:05/17/21 00:21:00 long stop loss:-14.77766171999997 revenue:178.7456 
*[89495] time:05/17/21 04:50:00 long stop loss:-15.043513399999995 revenue:179.8917 
*[89989] time:05/17/21 13:04:00 long stop loss:-15.455526959999986 revenue:189.1732 
*[90335] time:05/17/21 18:49:00 short stop loss:-15.76507976000002 revenue:190.3621
*[91722] time:05/18/21 17:55:00 short stop loss:-14.790404200000012 revenue:203.3047
*[91875] time:05/18/21 20:28:00 short stop loss:-14.96138812000001 revenue:199.831